In [43]:
llema_file_path = "inference_sat_math_results/llemma_math_sat_answers_2.csv"

mistral_file_path = "inference_sat_math_results/Mistral_Inference_SAT_Dataset.csv"

llama_file_path = "inference_sat_math_results/the_inference_llama3_sat_math.csv"

qwen_file_path = "inference_sat_math_results/inference_QWEN_sat_math.csv"

def extract_numbers(s):
    normalized = re.sub(r'[^\d\s]', '', s)  
    return list(map(int, re.findall(r'\d+', normalized))) 
def compare_number_lists(list1, list2):
    return set(list1).issubset(set(list2))  # list1=correct solution

In [67]:
import pandas as pd
from datasets import load_dataset, DatasetDict, Dataset
import matplotlib.pyplot as plt
import re


sat_math_dict = load_dataset("knazarali3/group2_processed_sat_math_cot")
train_dataset = sat_math_dict["train"]
test_dataset = sat_math_dict["test"]
test_dataset_df = pd.DataFrame(test_dataset)
test_dataset_df["row_index"] = test_dataset_df.index

In [68]:

def format_llema_inference_sat_math():
    df = pd.read_csv(llema_file_path, index_col=0)
    llema_test_dataset= df.merge(test_dataset_df, left_on='Question Id', right_on='id')

    llema_test_dataset['Correct Final Solution'] = llema_test_dataset['answer'].apply(extract_numbers)

    llema_test_dataset["llema_is_correct"] = llema_test_dataset.apply(
        lambda row: row["Final Solution"] in row["Correct Final Solution"], axis=1
    )

    llema_test_dataset = llema_test_dataset.rename(columns={"Final Solution": "llema_solution"})

    llema_3_results = llema_test_dataset[["Question Id", "llema_is_correct", "llema_solution"]]
    return llema_3_results


def format_mistral_inference_sat_math():
    df = pd.read_csv(mistral_file_path)
    mistral_test_dataset= df.merge(test_dataset_df, left_on='ID', right_on='id')

    mistral_test_dataset['Final Solution Numbers'] = mistral_test_dataset['answer'].apply(extract_numbers)


    mistral_test_dataset['Run_1'] = mistral_test_dataset['Run_1'].astype(str)

    mistral_test_dataset['Mistral Model Solution Numbers'] = mistral_test_dataset['Run_1'].apply(extract_numbers)

    mistral_test_dataset['mistral_is_correct'] = [
        compare_number_lists(a, b) for a, b in zip(mistral_test_dataset['Final Solution Numbers'], mistral_test_dataset['Mistral Model Solution Numbers'])
    ]

    mistral_test_dataset = mistral_test_dataset.rename(columns={"ID": "Question Id"})

    mistral_results = mistral_test_dataset[["Question Id", "mistral_is_correct", "Mistral Model Solution Numbers"]]
    return mistral_results    

def format_llama_inference_sat_math():
    df = pd.read_csv(llama_file_path)
    llama_test_dataset= df.merge(test_dataset_df, left_on='ID', right_on='id')

    llama_test_dataset['Final Solution Numbers'] = llama_test_dataset['answer'].apply(extract_numbers)

    llama_test_dataset['Run_1'] = llama_test_dataset['Run_1'].astype(str)

    llama_test_dataset['Llama Model Solution Numbers'] = llama_test_dataset['Run_1'].apply(extract_numbers)

    llama_test_dataset['llama_is_correct'] = [
        compare_number_lists(a, b) for a, b in zip(llama_test_dataset['Final Solution Numbers'], llama_test_dataset['Llama Model Solution Numbers'])
    ]

    llama_test_dataset = llama_test_dataset.rename(columns={"ID": "Question Id"})

    llama_results = llama_test_dataset[["Question Id", "llama_is_correct", "Llama Model Solution Numbers"]]
    return llama_results

def format_qwen_inference_sat_math():
    df = pd.read_csv(qwen_file_path)
    qwen_test_dataset= df.merge(test_dataset_df, left_on='row_index', right_on='row_index')


    qwen_test_dataset['Final Solution Numbers'] = qwen_test_dataset['answer'].apply(extract_numbers)

    qwen_test_dataset['Run_1'] = qwen_test_dataset['Run_1'].astype(str)

    qwen_test_dataset['Qwen Model Solution Numbers'] = qwen_test_dataset['Run_1'].apply(extract_numbers)

    qwen_test_dataset['qwen_is_correct'] = [
        compare_number_lists(a, b) for a, b in zip(qwen_test_dataset['Final Solution Numbers'], qwen_test_dataset['Qwen Model Solution Numbers'])
    ]

    qwen_test_dataset = qwen_test_dataset.rename(columns={"id": "Question Id"})

    qwen_results = qwen_test_dataset[["Question Id", "qwen_is_correct", "Qwen Model Solution Numbers"]]

    return qwen_results

In [66]:
llama_results = format_llama_inference_sat_math()
llema_results = format_llema_inference_sat_math()
qwen_results = format_qwen_inference_sat_math()
mistral_results = format_mistral_inference_sat_math()

# Check where all the answers correct column is True
# i.e. all models got the answer correct
all_results = ((llama_results.merge(llema_results)).merge(qwen_results)).merge(mistral_results)
all_results["all_correct"] = (all_results["llama_is_correct"] == True) & (all_results["llema_is_correct"] == True) & (all_results["qwen_is_correct"] == True) & (all_results["mistral_is_correct"]==True)

all_results["all_correct"].value_counts()

all_correct
False    6042
True      152
Name: count, dtype: int64

In [69]:
llama_results["llama_is_correct"].value_counts()

llama_is_correct
True     3862
False    2428
Name: count, dtype: int64

In [70]:
llema_results["llema_is_correct"].value_counts()

llema_is_correct
False    5354
True      936
Name: count, dtype: int64

In [71]:
qwen_results["qwen_is_correct"].value_counts()

qwen_is_correct
False    3156
True     3134
Name: count, dtype: int64

In [72]:
mistral_results["mistral_is_correct"].value_counts()

mistral_is_correct
False    5168
True     1026
Name: count, dtype: int64